In [1]:
!pip install strsimpy
from strsimpy.jaro_winkler import JaroWinkler
from strsimpy.normalized_levenshtein import NormalizedLevenshtein

import pandas as pd
import numpy as np
import json
import fileinput
from tqdm import tqdm
import string
import re
import json



Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/anaconda3/envs/jaeyoung/bin/python -m pip install --upgrade pip' command.


In [2]:
who_player = pd.DataFrame()
with fileinput.input(files="whoscored_crawler/player_info_whoscored.jl") as file:
    for line in file:
        conv = json.loads(line)
        who_player = who_player.append(conv, ignore_index=True)
who_player.head()

,player_url,Name,Shirt Number,Age,Height,Nationality,Positions,Strengths,Weaknesses,Style_of_Play,Current Team
0,https://www.whoscored.com/Players/4384/Show/Di...,Didier Drogba,11,43 years old (11-03-1978),189cm,Ivory Coast,Forward,"[Long shots, Direct free-kicks, Finishing, Aer...","[Holding on to the ball, Passing, Discipline]","[Plays the ball off the ground often, Indirect...",NaN
1,https://www.whoscored.com/Players/8040/Show/Ce...,Cesc Fàbregas,4,34 years old (04-05-1987),180cm,Spain,Forward,"[Through balls, Taking set-pieces, Key passes,...","[Finishing, Tackling, Discipline]","[Plays the ball off the ground often, Does not...",Monaco
2,https://www.whoscored.com/Players/4564/Show/Ro...,Robin van Persie,10,38 years old (06-08-1983),183cm,Netherlands,Forward,"[Direct free-kicks, Long shots, Key passes, Dr...","[Tackling, Discipline, Defensive contribution]","[Likes to play short passes, Likes to dribble,...",NaN
3,https://www.whoscored.com/Players/109/Show/Luí...,Luís Boa Morte Pereira,35,44 years old (04-08-1977),178cm,Portugal,Forward,NaN,NaN,NaN,NaN
4,https://www.whoscored.com/Players/48/Show/Fran...,Frank Lampard,8,43 years old (20-06-1978),184cm,England,Midfielder (Centre),[Finishing],"[Tackling, Passing]","[Likes to do layoffs, Does not dive into tackles]",NaN


In [3]:
world_player = pd.DataFrame()
with fileinput.input(files="worldfootball_crawler/player_info_worldfootball.jl") as file:
    for line in file:
        conv = json.loads(line)
        world_player = world_player.append(conv, ignore_index=True)
world_player.head()

,team_history,player_url,name,Born,Placeofbirth,Nationality,Height,Weight,Position(s),Foot,Current_team_info,Homepage,sizeofshoe
0,"{'0': {'period': '07/2011 - 06/2012', 'team': ...",https://www.worldfootball.net/player_summary/m...,Michael Johnson,[24.02.1988],"[Urmston, England]",[England],[183cm],[79kg],[DefensiveMidfielder],[right],NaN,NaN,NaN
1,"{'0': {'period': '10/2017 - 06/2019', 'team': ...",https://www.worldfootball.net/player_summary/r...,Roger Johnson,[28.04.1983],"[Ashford, England]",[England],[191cm],[78kg],[CentreBack],[right],NaN,NaN,NaN
2,"{'1': {'period': '07/2020 - 08/2020', 'team': ...",https://www.worldfootball.net/player_summary/j...,Jake Jervis,[17.09.1991],"[Wolverhampton, England]",[England],[191cm],NaN,"[RightWinger, CentreForward]",[right],"{'team_name': 'SJK Seinäjoki', 'country': 'Fin...",NaN,NaN
3,"{'0': {'period': '01/2019 - 06/2019', 'team': ...",https://www.worldfootball.net/player_summary/p...,Paul Scharner,[11.03.1980],"[Scheibbs, Austria]",[Austria],[191cm],[81kg],"[CentreBack, DefensiveMidfielder]",[right],NaN,[www.paulscharner.at],NaN
4,"{'0': {'period': '05/2013 - 06/2013', 'team': ...",https://www.worldfootball.net/player_summary/d...,David Bentley,[27.08.1984],"[Peterborough, England]",[England],[175cm],[68kg],"[RightMidfielder, RightWinger]",[right],NaN,NaN,NaN


In [4]:
len(who_player)

2155

In [5]:
len(world_player)

2832

In [6]:
                       
len(who_player["player_url"].drop_duplicates())

2155

## ID, Map Generation

In [7]:
len(world_player["player_url"])

2832

In [8]:
len(world_player["player_url"].drop_duplicates())

2832

In [9]:
world_player.groupby("name").count()["player_url"].value_counts()

1    2812
2      10
Name: player_url, dtype: int64

In [10]:
id2player = dict()
player2id = dict()

for idx, player_url in enumerate(list(world_player["player_url"].drop_duplicates())):
    id2player[idx + 10**6] = player_url
    player2id[player_url] = idx + 10**6

for idx, player_url in enumerate(list(who_player["player_url"].drop_duplicates())):
    id2player[idx + 2*(10**6)] = player_url
    player2id[player_url] = idx + 2 * (10**6)

In [11]:
len(who_player["player_url"])

2155

In [12]:
len(who_player["player_url"].drop_duplicates())

2155

In [13]:
len(id2player)

4987

In [14]:
# id2who = dict()
# who2id = dict()

# for idx, player_url in enumerate(who_player["player_url"].drop_duplicates()):
#     id2who[idx + (10**6)*2] = player_url
#     who2id[player_url] = idx + (10**6)*2

## Blocking

In [15]:
def f(x):
    if type(x) == float:
        return 0
    elif len(x[0])<15:
        tmp = x[0].split(".")
        return int(str(tmp[2]) + str(tmp[1]) + str(tmp[0]))
    else:
        tmp = x[0].split(".")
        return int(str(tmp[2])[:4] + str(tmp[1]) + str(tmp[0]))
world_player["birth_block"] = world_player["Born"].apply(f)

In [16]:
who_player["DOB"] = who_player["Age"].apply(lambda x : x.split(" years old (")[1][:-1])

who_player["Age"] = who_player["Age"].apply(lambda x : x.split(" years old (")[0])

In [17]:
def f(x):
    tmp = x.split("-")
    return int(str(tmp[2]) + str(tmp[1]) + str(tmp[0]))
who_player["birth_block"] = who_player["DOB"].apply(f)

In [18]:
who_player['player_id'] = who_player["player_url"].apply(lambda x : player2id[x])
world_player['player_id'] = world_player["player_url"].apply(lambda x : player2id[x])

In [19]:
who_birth = who_player[["player_id", "birth_block"]]
world_birth = world_player[["player_id", "birth_block"]]
birth_df = who_birth.append(world_birth)

In [20]:
# ## player_Block_obs.txt
# with open(data_dir + "playerBlock_obs.txt", "w") as f:
#     for idx, row in birth_df.iterrows():
#         f.write(str(row["player_id"]) + "\t" + str(row["birth_block"]) + "\n")

In [21]:
# birth_df.groupby("birth_block").count().sort_values(by = "player_id", ascending = False)
birth_df

,player_id,birth_block
0,2000000,19780311
1,2000001,19870504
2,2000002,19830806
3,2000003,19770804
4,2000004,19780620
...,...,...
2827,1002827,20010203
2828,1002828,19911123
2829,1002829,19910411
2830,1002830,19840922


In [22]:
world_name = world_player[["player_id", "name"]]
who_name = who_player[["player_id", "Name"]].rename({"Name" : "name"}, axis =1 )
name_df = world_name.append(who_name)

In [23]:
# ## player_name_obs.txt
# with open(data_dir + "playerName_obs.txt", "w") as f:
#     for idx, row in name_df.iterrows():
#         f.write(str(row["player_id"]) + "\t" + str(row["name"]) + "\n")

In [24]:
name_df

,player_id,name
0,1000000,Michael Johnson
1,1000001,Roger Johnson
2,1000002,Jake Jervis
3,1000003,Paul Scharner
4,1000004,David Bentley
...,...,...
2150,2002150,Joel Mumbongo
2151,2002151,Tyrese Francois
2152,2002152,Bailey Peacock-Farrell
2153,2002153,Allan Tchaptchet


# Matching

In [25]:
tmp = pd.DataFrame(birth_df.groupby("birth_block")["player_id"].apply(list))
match_dict =dict()
match_dict2 = dict()
id2name = name_df.set_index('player_id').to_dict()["name"]
Lev = NormalizedLevenshtein().similarity

for idx, row in tmp.iterrows():
    if idx != 0 and len(row.player_id)>1:
        name_list_1 = []
        name_list_2 = []
        for idx in row.player_id:
            if str(idx)[0] == "1":
                name_list_1.append((idx,id2name[idx].lower()))
            if str(idx)[0] == "2":
                name_list_2.append((idx,id2name[idx].lower()))
        if len(name_list_1) == 0 or len(name_list_2) == 0:
            continue
        else:
            for (idx1, name1) in name_list_1:
                for (idx2, name2) in name_list_2:
                    if Lev(name1, name2) > 0.4:
#                         if Lev(name1, name2) < 0.5:
#                             print(name1 +"\t" + name2 + "\t" + str(Lev(name1, name2)))
                        try:
                            a = match_dict[idx1]
                            if Lev(name1, name2) > Lev(name1, a[0][1]):
                                match_dict[idx1] = [(idx2, name2)]
                                match_dict2[idx2] = [(idx1, name1)]
                        except:
                            match_dict[idx1] = match_dict.get(idx1, []) + [(idx2, name2)]
                            match_dict2[idx2] = match_dict2.get(idx2, []) + [(idx1, name1)]
        
            
                        
#                         match_dict[idx1] = match_dict.get(idx1, []) + [(idx2, name2)]
#                         match_dict2[idx2] = match_dict2.get(idx2, []) + [(idx1, name1)]
        
        
                        

In [26]:
id_list1 = [i for i in list(id2player.keys()) if str(i)[0] == "1" and i in match_dict.keys()]

In [27]:
id_list2 = [i for i in list(id2player.keys()) if str(i)[0] == "2" and i in match_dict2.keys()]

In [28]:
from collections import Counter

In [29]:
tmp = []
for i in list(match_dict.values()):
    tmp+=i

In [30]:
[i for i in Counter(tmp).items() if i[1] == 2]

[((2001079, 'marc muniesa'), 2),
 ((2001668, 'charlie taylor'), 2),
 ((2001329, 'josé pozo'), 2),
 ((2001835, 'ryan sessegnon'), 2)]

In [31]:
tmp = []
for i in list(match_dict2.values()):
    tmp+=i

In [32]:
[i for i in Counter(tmp).items() if i[1] == 2]

[((1000504, 'jermaine jenas'), 2),
 ((1000994, 'marcus olsson'), 2),
 ((1000569, 'kieran gibbs'), 2),
 ((1000972, 'michael keane'), 2),
 ((1002212, 'jacob murphy'), 2)]

In [33]:
len(id_list1)

1945

In [34]:
len(id_list2)

1941

In [35]:
len(match_dict)

1945

In [36]:
[i for i in match_dict2.items() if len(i[1])!=1]

[(2000222, [(1000344, 'tyrone mears'), (1000504, 'jermaine jenas')]),
 (2000207, [(1000473, 'martin olsson'), (1000994, 'marcus olsson')]),
 (2000018, [(1000036, 'ciaran clark'), (1000569, 'kieran gibbs')]),
 (2001079, [(1001342, 'muniesa'), (1002597, 'mark gillespie')]),
 (2000872, [(1000971, 'will keane'), (1000972, 'michael keane')]),
 (2001668, [(1001100, 'charles banya'), (1002217, 'charlie taylor')]),
 (2001187, [(1001343, 'josh murphy'), (1002212, 'jacob murphy')]),
 (2001329, [(1001527, 'josé pozo'), (1001741, 'josh doherty')]),
 (2001835, [(1002401, 'steven sessegnon'), (1002404, 'ryan sessegnon')])]

In [37]:
k = [i for i in match_dict.items() if len(i[1])!=1]
k

[]

In [38]:
len(match_dict)

1945

In [39]:
len(who_player["player_url"].drop_duplicates())

2155

In [40]:
len(world_player["player_url"].drop_duplicates())

2832

In [41]:
len(player2id)

4987

In [42]:
def f(x):
    try:
        return match_dict[x][0][0]
    except:
        return "No match"
world_player["match_player_id"] = world_player["player_id"].apply(f)

In [43]:
# outer join in python pandas
outer_join_df=pd.merge(world_player, who_player, left_on = "match_player_id", right_on = "player_id", how='outer')
outer_join_df


,team_history,player_url_x,name,Born,Placeofbirth,Nationality_x,Height_x,Weight,Position(s),Foot,...,Height_y,Nationality_y,Positions,Strengths,Weaknesses,Style_of_Play,Current Team,DOB,birth_block_y,player_id_y
0,"{'0': {'period': '07/2011 - 06/2012', 'team': ...",https://www.worldfootball.net/player_summary/m...,Michael Johnson,[24.02.1988],"[Urmston, England]",[England],[183cm],[79kg],[DefensiveMidfielder],[right],...,183cm,England,Midfielder,NaN,NaN,NaN,NaN,24-02-1988,19880224.0,2000534.0
1,"{'0': {'period': '10/2017 - 06/2019', 'team': ...",https://www.worldfootball.net/player_summary/r...,Roger Johnson,[28.04.1983],"[Ashford, England]",[England],[191cm],[78kg],[CentreBack],[right],...,191cm,England,Defender (Centre),[Blocking the ball],[Passing],"[Indirect set-piece threat, Likes to play long...",NaN,28-04-1983,19830428.0,2000086.0
2,"{'1': {'period': '07/2020 - 08/2020', 'team': ...",https://www.worldfootball.net/player_summary/j...,Jake Jervis,[17.09.1991],"[Wolverhampton, England]",[England],[191cm],NaN,"[RightWinger, CentreForward]",[right],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'1': {'period': '07/2016 - 06/2019', 'team': ...",https://www.worldfootball.net/player_summary/j...,Jake Bidwell,[21.03.1993],"[Southport, England]",[England],[183cm],[80kg],"[LeftBack, LeftMidfielder]",[left],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'0': {'period': '07/2013 - 06/2016', 'team': ...",https://www.worldfootball.net/player_summary/b...,Bosingwa,[24.08.1982],"[Mbandaka, CongoDR]","[Portugal, CongoDR]",[183cm],[75kg],"[RightBack, RightMidfielder]",[right],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,190cm,Brazil,Defender (Centre),[Passing],[],"[Likes to play short passes, Commits fouls often]",Arsenal,19-12-1997,19971219.0,2002066.0
3042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,178cm,Brazil,Midfielder (Left),"[Holding on to the ball, Tackling, Concentration]",[],[Commits fouls often],Wolverhampton Wanderers,19-02-1989,19890219.0,2002098.0
3043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,190cm,NaN,Forward,"[Long shots, Defensive contribution]","[Finishing, Aerial Duels, Holding on to the ball]",[Likes to dribble],Burton,13-08-2003,20030813.0,2002099.0
3044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,175cm,Croatia,"Midfielder (Centre, Left)",[Tackling],[],"[Gets fouled often, Likes to do layoffs, Likes...",Hajduk Split,29-08-1995,19950829.0,2002105.0


In [44]:
outer_join_df.isna().sum()

team_history          214
player_url_x          214
name                  214
Born                  216
Placeofbirth          214
Nationality_x         214
Height_x              267
Weight                374
Position(s)           314
Foot                  471
Current_team_info    1275
Homepage             2997
sizeofshoe           3030
birth_block_x         214
player_id_x           214
match_player_id       214
player_url_y          887
Name                  887
Shirt Number          989
Age                   887
Height_y              921
Nationality_y         917
Positions             887
Strengths            1168
Weaknesses           1168
Style_of_Play        1168
Current Team         1721
DOB                   887
birth_block_y         887
player_id_y           887
dtype: int64

In [45]:
outer_join_df.to_csv("player_join.csv")

In [46]:
##############

In [91]:
## Name sim
Lev = NormalizedLevenshtein().similarity
with open(data_dir + "simName_obs.txt", "w") as f:
    tmp = pd.DataFrame(birth_df.groupby("birth_block")["player_id"].apply(list))
    for id_list in tmp["player_id"]:
        for idx1 in id_list:
            for idx2 in id_list:
                name1 = str(list(name_df[name_df["player_id"] == int(idx1)]["name"])[0])
                name2 = str(list(name_df[name_df["player_id"] == int(idx2)]["name"])[0])
                f.write(name1 + "\t" + name2 + "\t" + str(Lev(name1.lower(), name2.lower() )) + "\n")

with open(data_dir + "simName_obs.txt", "r") as f:
    simName = f.read()

with open(data_dir + "simName_obs.txt", "w") as f:
    for i in set(simName.split("\n")):
        if i != "":
            f.write(i + "\n")

In [89]:
## target
with open(data_dir + "same_player_target.txt", "w") as f:
    tmp = pd.DataFrame(birth_df.groupby("birth_block")["player_id"].apply(list))
    for id_list in tmp["player_id"]:
        for idx1 in id_list:
            for idx2 in id_list:
                f.write(str(int(idx1)) + "\t" + str(int(idx2)) + "\n")
                
with open(data_dir + "same_player_target.txt", "r") as f:
    data = f.read()

with open(data_dir + "same_player_target.txt", "w") as f:
    for i in set(data.split("\n")):
        if i != "":
            f.write(i + "\n")



In [95]:
id2player[1001878]

'https://www.worldfootball.net/player_summary/steven-pienaar/'

In [94]:
id2player[2000048]

'https://www.whoscored.com/Players/3553/Show/Steven-Pienaar'

In [92]:
birth_df[birth_df["player_id"] == 1001878]

,player_id,birth_block
1878,1001878,19820317


In [93]:
birth_df[birth_df["player_id"] == 2000048]

,player_id,birth_block
48,2000048,19820317


In [18]:
# tmp = world_player[["name","player_url"]]
world_player["player_id"] = world_player["player_url"].apply(lambda x : player2id[x])
who_player["player_id"] = who_player["player_url"].apply(lambda x : player2id[x])


In [20]:
## playerName_obs.txt
with open("PSL_data/playerName_obs.txt", "w") as f:
    for idx, player in id2player.items():
        f.write(str(idx) + "\t" + player + "\n")

In [21]:
## player_Block_obs.txt
with open("PSL_data/playerBlock_obs.txt", "w") as f:
    tmp = world_player[["player_id", "name"]].drop_duplicates()
    for idx, row in tmp.iterrows():
        f.write(str(row["player_id"]) + "\t" + row["name"] + "\n")
    tmp = who_player[["player_id", "player_block"]].drop_duplicates()
    for idx, row in tmp.iterrows():
        f.write(str(row["player_id"]) + "\t" + row["player_block"] + "\n")

In [22]:
with open("PSL_data/playerBlock_obs.txt", "r") as f:
    dt = f.read()

In [23]:
### one player more than two block...
tmp = pd.DataFrame(pd.DataFrame([i.split("\t") for i in dt.split("\n")])[:-1].groupby(0)[1].apply(list))

In [24]:
tmp["l"] = tmp[1].apply(len)

In [25]:
list(tmp[tmp["l"] != 1][1])

[['Bernard Diomède', 'Bernard'],
 ['Bojan Djordjic', 'Bojan'],
 ['Bojan Djordjic', 'Bojan'],
 ['Wesley Boyle', 'Wesley'],
 ['Adrian Cáceres', 'Adrián'],
 ['Bernard Lambourde', 'Bernard'],
 ['Lauren', 'Allan'],
 ['Émerson Thome', 'Emerson'],
 ['Méndez', 'Nathaniel Mendez-Laing'],
 ['Méndez', 'Álvaro'],
 ['Méndez', 'Gabriel'],
 ['Méndez', 'Juan'],
 ['Méndez', 'Marcelo'],
 ['Méndez', 'Fernando'],
 ['Méndez', 'Oscar'],
 ['Méndez', 'Gabriel'],
 ['Méndez', 'Eduardo'],
 ['Méndez', 'Joel'],
 ['Olivier Bernard', 'Bernard'],
 ['Marcelino', 'Núñez'],
 ['Alex Rae', 'Alex'],
 ['Alex Nyarko', 'Alex'],
 ['John Collins', 'Collins John'],
 ['John Collins', 'Collins John'],
 ['John Collins', 'Collins John'],
 ['Allan Johnston', 'Allan'],
 ['Juan', 'Juan Iturbe'],
 ['Juan', 'Juan Cuadrado'],
 ['Juan', 'Mata'],
 ['Juan', 'Cabral'],
 ['Juan', 'Núñez'],
 ['Ricardo Gardner', 'Ricardo'],
 ['Joel Kitamirike', 'Joel'],
 ['Bo Hansen', 'Martin Hansen'],
 ['Bruno Cheyrou', 'Bruno'],
 ['Gilberto Silva', 'Gilberto']

In [26]:
len(set([i.split("\t")[0] for i in dt.split("\n")]))

15080

In [27]:
world_player[['player_id', 'Born', 'Placeofbirth']]

,player_id,Born,Placeofbirth
0,1000000,[03.02.1967],"[Kenilworth, England]"
1,1000001,[29.08.1976],"[Dublin, Ireland]"
2,1000002,[30.11.1983],"[Sydney, Australia]"
3,1000003,[18.11.1980],"[Cambridge, England]"
4,1000004,[28.02.1974],"[Birmingham, England]"
...,...,...,...
4355,1004355,[08.11.2001],"[Petersfield, England]"
4356,1004356,[03.02.2001],"[Preston, England]"
4357,1004357,[08.10.1997],"[Poole, England]"
4358,1004358,[23.11.1991],"[PortoCalvo, Brazil]"


In [28]:
def f(x):
    if type(x) == float:
        return 0
    else:
        return len(x[0])
list(world_player[world_player["Born"].apply(f)>15]["player_url"])

['https://www.worldfootball.net/player_summary/ugo-ehiogu/',
 'https://www.worldfootball.net/player_summary/gary-speed/',
 'https://www.worldfootball.net/player_summary/dean-richards/',
 'https://www.worldfootball.net/player_summary/jlloyd-samuel/',
 'https://www.worldfootball.net/player_summary/jimmy-davis/',
 'https://www.worldfootball.net/player_summary/alan-miller/',
 'https://www.worldfootball.net/player_summary/goran-bunjevcevic/',
 'https://www.worldfootball.net/player_summary/marc-vivien-foe/',
 'https://www.worldfootball.net/player_summary/peter-whittingham/',
 'https://www.worldfootball.net/player_summary/reyes/',
 'https://www.worldfootball.net/player_summary/papa-bouba-diop/',
 'https://www.worldfootball.net/player_summary/vaclav-drobny/',
 'https://www.worldfootball.net/player_summary/marton-fueloep/',
 'https://www.worldfootball.net/player_summary/liam-miller/',
 'https://www.worldfootball.net/player_summary/valery-mezague/',
 'https://www.worldfootball.net/player_summary

In [29]:
def f(x):
    if type(x) == float:
        return 0
    elif len(x[0])<15:
        tmp = x[0].split(".")
        return int(str(tmp[2]) + str(tmp[1]) + str(tmp[0]))
    else:
        tmp = x[0].split(".")
        return int(str(tmp[2])[:4] + str(tmp[1]) + str(tmp[0]))
world_player['Born'] = world_player['Born'].apply(f)

In [30]:
def f(x):
    if type(x) == float:
        return 0
    if x == None:
        return 0
    else:
        tmp = x.split("-")
        return int(str(tmp[2]) + str(tmp[1]) + str(tmp[0]))


who_player["DOB"] = who_player["DOB"].apply(f)

In [31]:
tmp1 = world_player[["player_id", "Born"]]
tmp1 = tmp1.rename({"Born" : "DOB"}, axis = 1)
tmp2 = who_player[["player_id", "DOB"]]

In [32]:
dob = tmp1.append(tmp2)

In [33]:
birth_id2birth = dict()
birth2birth_id = dict()
for idx, birth in enumerate(dob["DOB"].drop_duplicates()):
    birth_id2birth[idx] = birth
    birth2birth_id[birth] = idx

In [34]:
dob["birth_id"] = dob["DOB"].apply(lambda x : birth2birth_id[x])

In [35]:
## DOBName_obs.txt
tmp = dob[["birth_id", "DOB"]].drop_duplicates()
with open("PSL_data/birthName_obs.txt", "w") as f:
    for idx, row in tmp.iterrows():
        f.write(str(row.birth_id) + "\t" + str(row["DOB"]) + "\n")

In [52]:
## DOBof_obs.txt
with open("PSL_data/ofDOB_obs.txt", "w") as f:
    for idx, row in dob.drop_duplicates().iterrows():
#         f.write(str(row.player_id) + "\t" + str(row.birth_id) + "\n")
        f.write(str(row.birth_id) + "\t" + str(row.player_id) + "\n")


In [37]:
##simName_obs.txt
with open("PSL_data/playerBlock_obs.txt", "r") as f:
    players = f.read()

In [38]:
block = pd.DataFrame([i.split("\t") for i in players.split("\n")][:-1], columns = ["id", "block"])

In [39]:
# list(who_player[who_player["player_id"] == int(idx2)]["Name"])[0]

In [40]:
total_player = who_player.rename({"Name" : "name"}, axis = 1).append(world_player)

In [42]:
Lev = NormalizedLevenshtein().similarity
with open("PSL_data/simName_obs.txt", "w") as f:
    tmp = pd.DataFrame(block.groupby("block")["id"].apply(list))
    for id_list in tmp["id"]:

        for idx1 in id_list:
            for idx2 in id_list:
                f.write(str(int(idx1)) + "\t" + str(int(idx2)) + "\t" + str(Lev(list(total_player[total_player["player_id"] == int(idx2)]["name"])[0], list(total_player[total_player["player_id"] == int(idx1)]["name"])[0]) ) + "\n")



In [43]:
simName = pd.read_csv("PSL_data/simName_obs.txt", header = None, sep = "\t")
simName = simName.drop_duplicates()
with open("PSL_data/simName_obs.txt", "w") as f:
    for idx, row in simName.iterrows():
        f.write(str(int(row[0])) + "\t" + str(int(row[1])) + "\t" + str(row[2]) + "\n")

In [44]:
##simDOB_obs.txt
def g(a, b):
    if a == 1562 or b == 1562:
        return 0.5
    elif a == b:
        return 1.0
    else:
        return 0.0

In [45]:
with open("PSL_data/simDOB_obs.txt", "w") as f:
    tmp = pd.DataFrame(block.groupby("block")["id"].apply(list))
    for id_list in tmp["id"]:

        for idx1 in id_list:
            for idx2 in id_list:
                birth_id1 = int(list(dob[dob["player_id"]==int(idx1)]["birth_id"])[0])
                birth_id2 = int(list(dob[dob["player_id"]==int(idx2)]["birth_id"])[0])
                f.write(str(birth_id1) + "\t" + str(birth_id2) + "\t" + str(g(birth_id1, birth_id2)) + "\n")


In [46]:
simdob = pd.read_csv("PSL_data/simDOB_obs.txt", header = None, sep = "\t")

In [47]:
simdob = simdob.drop_duplicates()

In [48]:
simdob

,0,1,2
0,1625,1625,1.0
1,1625,81,0.0
3,1625,1560,0.0
4,81,1625,0.0
5,81,81,1.0
...,...,...,...
973968,3390,3390,1.0
973972,312,312,1.0
973976,234,234,1.0
973978,1546,1546,1.0


In [49]:
with open("PSL_data/simDOB_obs.txt", "w") as f:
    for idx, row in simdob.iterrows():
        f.write(str(int(row[0])) + "\t" + str(int(row[1])) + "\t" + str(row[2]) + "\n")
    

In [50]:
####

In [62]:
with open("PSL_data/samePlayer_target_obs.txt", "w") as f:
    tmp = pd.DataFrame(block.groupby("block")["id"].apply(list))
    for id_list in tmp["id"]:
        if len(id_list)>=2:
            list_1 = []
            list_2 = []
            for idx in id_list:
                if idx[0] == '1':
                    list_1.append(idx)
                if idx[0] == '2':
                    list_2.append(idx)

            for idx1 in list_1:
                for idx2 in list_2:
                    f.write(idx1 + "\t" + idx2 + "\n" )


In [56]:
id_list

['1001832', '2003648', '2003649', '2003650']

In [60]:
id_list[3:]

['2003650']

In [63]:
tmp = pd.DataFrame(block.groupby("block")["id"].apply(list))

In [64]:
tmp

,id
block,
Aaron Brown,"[1001832, 2003648, 2003649, 2003650]"
Aaron Connolly,"[1003638, 2008303, 2008304, 2008305]"
Aaron Cresswell,"[1003072, 2007126]"
Aaron Doran,"[1001982, 2003803, 2003804]"
Aaron Hughes,"[1000475, 2000606]"
...,...
İlkay Gündoğan,[1003513]
Ľubomir Michalík,[1001734]
Ľubomír Šatka,[1002940]


In [53]:
who_player.isna().sum()

player_block        0
player_url          0
Name                0
Shirt Number     6602
Age              1161
Height           3753
Nationality         0
Positions        1609
Strengths        8087
Weaknesses       8087
Style_of_Play    8087
Current Team     8527
DOB                 0
player_id           0
dtype: int64

In [69]:
who_player[who_player["DOB"] == 0]["player_url"][23]


'https://www.whoscored.com/Players/339830/Show/Ben-Clark'

In [84]:
tmp = who_player[["player_block", "player_url"]]
tmp["player_id"] = tmp["player_url"].apply(lambda x : who2id[x])

/Users/jaeyoungkim/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [89]:
who_player

,player_block,player_url,Name,Shirt Number,Age,Height,Nationality,Positions,Strengths,Weaknesses,Style_of_Play,Current Team,DOB
0,David Carney,https://www.whoscored.com/Players/13253/Show/D...,David Carney,17,37,181cm,Australia,Defender (Left),[],"[Concentration, Aerial Duels, Passing, Holding...","[Plays the ball off the ground often, Likes to...",NaN,30-11-1983
1,Stephen Carr,https://www.whoscored.com/Players/1354/Show/St...,Stephen Carr,2,45,175cm,Ireland,Defender (Right),NaN,NaN,NaN,NaN,29-08-1976
2,Jamie Carragher,https://www.whoscored.com/Players/1329/Show/Ja...,Jamie Carragher,23,43,185cm,England,Defender (Centre),[Passing],"[Aerial Duels, Tackling]",[Does not dive into tackles],NaN,28-01-1978
3,Michael Carrick,https://www.whoscored.com/Players/2115/Show/Mi...,Michael Carrick,16,40,188cm,England,Midfielder (Centre),"[Passing, Concentration]",[],"[Likes to shoot from distance, Does not dive i...",NaN,28-07-1981
4,Lee Carsley,https://www.whoscored.com/Players/114/Show/Lee...,Lee Carsley,26,47,175cm,Ireland,Midfielder (Centre),NaN,NaN,NaN,NaN,28-02-1974
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10923,George Wickens,https://www.whoscored.com/Players/363440/Show/...,George Wickens,NaN,None,NaN,England,NaN,NaN,NaN,NaN,Wealdstone,None
10924,Thiago Silva,https://www.whoscored.com/Players/28550/Show/T...,Thiago Emiliano da Silva,6,37,183cm,Brazil,Defender (Centre),"[Passing, Concentration]","[Discipline, Tackling]",[Does not dive into tackles],Chelsea,22-09-1984
10925,Thiago Silva,https://www.whoscored.com/Players/392670/Show/...,Thiago Silva,NaN,None,NaN,Brazil,NaN,NaN,NaN,NaN,NaN,None
10926,Thiago Silva,https://www.whoscored.com/Players/91117/Show/T...,Thiago da Silva,NaN,38,174cm,Brazil,Midfielder,NaN,NaN,NaN,NaN,22-06-1983


In [88]:
tmp[tmp["player_id"] == 1242]

,player_block,player_url,player_id
1243,Bruno Cheyrou,https://www.whoscored.com/Players/1445/Show/Br...,1242
8177,Bruno,https://www.whoscored.com/Players/1445/Show/Br...,1242


In [91]:
tmp["player_id"].value_counts()

1242     2
2307     2
3858     2
5685     2
3683     2
        ..
3634     1
3635     1
3636     1
3637     1
10718    1
Name: player_id, Length: 10719, dtype: int64

In [77]:
# a = JaroWinkler().similarity
a = NormalizedLevenshtein().similarity

In [112]:
pd.read_csv("PSL_data/birthName_obs.txt", header = None, sep ="\t")

,0,1
0,0,19760829
1,1,19831130
2,2,19670203
3,3,19740228
4,4,19700730
...,...,...
6326,6326,19910804
6327,6327,19890824
6328,6328,19890430
6329,6329,20020407


In [129]:
ofdob = pd.read_csv("PSL_data/ofDOB_obs.txt", header = None, sep ="\t")

In [131]:
ofdob.drop_duplicates()

,0,1
0,1000000,0
1,1000001,1
2,1000002,2
3,1000003,3
4,1000004,4
...,...,...
15279,2010714,1562
15280,2010715,2456
15281,2010716,4329
15282,2010717,1562


In [117]:
pd.read_csv("PSL_data/playerName_obs.txt", header = None, sep ="\t")


,0,1
0,1000000,https://www.worldfootball.net/player_summary/s...
1,1000001,https://www.worldfootball.net/player_summary/d...
2,1000002,https://www.worldfootball.net/player_summary/t...
3,1000003,https://www.worldfootball.net/player_summary/l...
4,1000004,https://www.worldfootball.net/player_summary/p...
...,...,...
15075,2010714,https://www.whoscored.com/Players/357131/Show/...
15076,2010715,https://www.whoscored.com/Players/358777/Show/...
15077,2010716,https://www.whoscored.com/Players/375312/Show/...
15078,2010717,https://www.whoscored.com/Players/376166/Show/...


In [121]:
pd.read_csv("PSL_data/playerBlock_obs.txt", header = None, sep ="\t")


,0,1
0,1000000,Stephen Carr
1,1000001,David Carney
2,1000002,Tim Flowers
3,1000003,Lee Carsley
4,1000004,Patrice Carteron
...,...,...
15284,2010718,George Wickens
15285,2010600,Thiago Silva
15286,2010707,Thiago Silva
15287,2010712,Thiago Silva


In [128]:
id2player[2004159]

'https://www.whoscored.com/Players/77308/Show/Fabio-Silva'

In [127]:
birth_id2birth[1105]

19830913

In [58]:
##simName_obs.txt
with open("PSL_data/samePlayer_target_obs.txt", "r") as f:
    targets = f.read()

In [61]:
id2player[2000256]

'https://www.whoscored.com/Players/265/Show/Lauren'

In [62]:
id2player[2000263]

'https://www.whoscored.com/Players/350282/Show/Lauren-Luke-Junior'